# Example - base on the example of wikipedia

## 1.HMM类的实现

In [3]:
class HMM:
    '''HMM类实现'''
    def __init__(self, initVectorPi, transitionMatrix, emissionMatrix):
        self.initVectorPi = initVectorPi
        self.transitionMatrix = transitionMatrix
        self.emissionMatrix = emissionMatrix
    
    @property #返回隐藏状态数目
    def statesNumber(self):
        return self.transitionMatrix.shape[0]


## 2.wikipedia的例子

In [84]:
#Wikipedia中的例子：https://en.wikipedia.org/wiki/Viterbi_algorithm#Example
############################################
'''
这里用数字代表字母
隐藏状态：0=Healthy, 1=Fever
观察状态：0=Normal, 1=Cold, 2=Dizzy
'''
hiddenStates=['Healthy','Fever']
observStates=['Normal','Cold','Dizzy']

wiki_initVectorPi1 = np.array([0.6, 0.4])#初始概率向量pi
wiki_transitionMatrix1 = np.array([[0.7, 0.3],#转移矩阵
                                  [0.4, 0.6]]) 
wiki_emissionMatrix1 = np.array([[0.5, 0.4, 0.1], #发射矩阵（或混淆矩阵）
                                [0.1, 0.3, 0.6]]) 
wiki_observations1 = [0, 1, 2] # 观察序列：Normal、Cold、Dizzy

wiki_hmm1 = HMM(wiki_initVectorPi1, wiki_transitionMatrix1, wiki_emissionMatrix1)
############################################

## 3.viterbi算法实现

In [85]:
import numpy as np

def viterbi(hmm, observations):
    '''Viterbi算法实现
    输入：HMM模型（包含初始向量pi，转移矩阵A、发射矩阵B）、观察序列
    输出：符合输入观察序列的最佳隐藏状态序列，储存每个时刻每个状态的概率值的矩阵
    '''
    #计算t=1时刻（从1开始计数）的局部概率 /delta_1(i) 
    allProbs = np.zeros((hmm.statesNumber,len(observations))) #用于存储每个阶段的概率值
    probs = hmm.emissionMatrix[:, observations[0]] * hmm.initVectorPi #对应元素相乘
    allProbs[:,0] = probs
    probs = probs.reshape(-1,1) #将行向量转换成列向量
    stack = [] #用来储存反向指针，即回溯得到最佳隐藏序列时会用到
    
    #计算t>1时刻的局部概率 /delta_t(i)
    t=1
    for obs in observations[1:]:
        #对应元素相乘，得到上一个时刻的局部概率 /delta_{t-1}(i) 和转移概率 a_{ij} 的对应乘积
        transProbs = hmm.transitionMatrix * probs
        #对于每一列（每个隐藏状态），找出使得 当前隐藏状态最大概率发生 的上一个时刻的隐藏状态的数字下标
        maxProbsIndex = np.argmax(transProbs, axis=0)
        #将反向指针压入栈
        stack.append(maxProbsIndex)
        #更新当前时间的局部概率
        probs = hmm.emissionMatrix[:, obs] * transProbs[maxProbsIndex, np.arange(hmm.statesNumber)]
        allProbs[:,t] = probs
        probs = probs.reshape(-1,1) #将行向量转换成列向量
        t+=1

    stateSeq = [np.argmax(probs)] #找出最大概率对应隐藏状态的下标
    
    #反向回溯
    while stack:
        #得到当前栈顶元素，并将该元素从栈顶去除
        maxProbsIndex = stack.pop()
        #依次将使得后一个时刻最大概率发生的隐藏状态添加到stateSeq中
        stateSeq.append(maxProbsIndex[stateSeq[-1]]) 

    stateSeq.reverse() #反转得到按时刻从早到晚的 最佳隐藏状态序列
    
    return stateSeq,allProbs

seq, probs = viterbi(wiki_hmm1, wiki_observations1) # 调用算法

############   结果输出  #############
print(' '*8 + " ".join(("%10s" % obs) for obs in observStates)) #输出所有观察状态
#输出每个阶段每个状态的概率值
for i in range(len(hiddenStates)):
    print("%8s:" % hiddenStates[i] + " ".join(("%10f" % prob) for prob in probs[i]))
#输出最佳隐藏序列 以及 对应的最高概率
print("The most possiblehidden state sequence is " , [hiddenStates[i] for i in seq], \
      "with highest probability of %8f" % max(probs[:,-1]))
############   结果输出  #############

            Normal       Cold      Dizzy
 Healthy:  0.300000   0.084000   0.005880
   Fever:  0.040000   0.027000   0.015120
The most possiblehidden state sequence is  ['Healthy', 'Healthy', 'Fever'] with highest probability of 0.015120


可见，输出最佳隐藏序列与wikipedia中的答案（如下）一致。

```
$ python viterbi_example.py
         0          1          2
Healthy: 0.30000 0.08400 0.00588
Fever: 0.04000 0.02700 0.01512
The steps of states are Healthy Healthy Fever with highest probability of 0.01512
```

## 4.前向-后向算法实现

In [46]:
import numpy as np

def forward(hmm,observations):
    '''前向算法实现
    输入：HMM模型（包含初始向量pi，转移矩阵A、发射矩阵B）、观察序列
    输出：输入观察序列在该HMM模型发生的概率、中间概率矩阵(alpha)
    '''
    rowNum = hmm.statesNumber
    colNum = len(observations)
    alpha = np.zeros((rowNum,colNum)) #二维矩阵，储存 T个时刻的alpha值
    
    #求t=1（t从1开始计数）时刻的alpha,即是 初始的概率与对应发射概率相乘
    alpha[:,0] = hmm.initVectorPi * np.transpose(hmm.emissionMatrix[:,observations[0]]) 
    
    #求 t=2,...,T 的alpha值
    for t in range(1,colNum):          
        for j in range(rowNum): 
            #求和符号可用点乘实现
            alpha[j,t] = hmm.emissionMatrix[j,observations[t]] * np.dot(alpha[:,t-1],hmm.transitionMatrix[:,j])
    #对最后一列alpha值求和
    ans=sum(alpha[:,colNum-1])
    return ans,alpha

def backward(hmm,observations):
    '''后向算法实现
    输入：HMM模型（包含初始向量pi，转移矩阵A、发射矩阵B）、观察序列
    输出：输入观察序列在该HMM模型发生的概率、中间概率矩阵(beta)
    '''    
    rowNum = hmm.statesNumber
    colNum = len(observations)
    beta = np.zeros((rowNum,colNum)) #二维矩阵，储存 T个时刻的beta值
    #t=T时，每一个元素赋值为1
    beta[:,colNum-1] = 1                  
    #求 t<T 时的beta值
    for t in reversed(range(colNum-1)):
        for i in range(rowNum):
            beta[i,t] = np.sum(beta[:,t+1] * hmm.transitionMatrix[i,:] * hmm.emissionMatrix[:,observations[t+1]])
    #对第一列beta值求和
    ans = np.sum(hmm.initVectorPi * beta[:,0] * hmm.emissionMatrix[:,observations[0]])
    return ans,beta

print(forward(wiki_hmm1, wiki_observations1)[0])
print(backward(wiki_hmm1, wiki_observations1)[0])

0.03628
0.03628


可见，前向算法和后向算法输出的结果是一样的。

## 5.Baum Welch算法实现

In [ ]:
import numpy as np

def forward(hmm,observations):
    '''前向算法实现
    输入：HMM模型（包含初始向量pi，转移矩阵A、发射矩阵B）、观察序列
    输出：输入观察序列在该HMM模型发生的概率、中间概率矩阵(alpha)
    '''
    rowNum = hmm.statesNumber
    colNum = len(observations)
    alpha = np.zeros((rowNum,colNum)) #二维矩阵，储存 T个时刻的alpha值
    
    #求t=1（t从1开始计数）时刻的alpha,即是 初始的概率与对应发射概率相乘
    alpha[:,0] = hmm.initVectorPi * np.transpose(hmm.emissionMatrix[:,observations[0]]) 
    
    #求 t=2,...,T 的alpha值
    for t in range(1,colNum):          
        for j in range(rowNum): 
            #求和符号可用点乘实现
            alpha[j,t] = hmm.emissionMatrix[j,observations[t]] * np.dot(alpha[:,t-1],hmm.transitionMatrix[:,j])
    #对最后一列alpha值求和
    ans=sum(alpha[:,colNum-1])
    return ans,alpha

def backward(hmm,observations):
    '''后向算法实现
    输入：HMM模型（包含初始向量pi，转移矩阵A、发射矩阵B）、观察序列
    输出：输入观察序列在该HMM模型发生的概率、中间概率矩阵(beta)
    '''    
    rowNum = hmm.statesNumber
    colNum = len(observations)
    beta = np.zeros((rowNum,colNum)) #二维矩阵，储存 T个时刻的beta值
    #t=T时，每一个元素赋值为1
    beta[:,colNum-1] = 1                  
    #求 t<T 时的beta值
    for t in reversed(range(colNum-1)):
        for i in range(rowNum):
            beta[i,t] = np.sum(beta[:,t+1] * hmm.transitionMatrix[i,:] * hmm.emissionMatrix[:,observations[t+1]])
    #对第一列beta值求和
    ans = np.sum(hmm.initVectorPi * beta[:,0] * hmm.emissionMatrix[:,observations[0]])
    return ans,beta

def getGamma(hmm,alpha,beta):
    '''计算gamma矩阵
    输入：HMM模型（包含初始向量pi，转移矩阵A、发射矩阵B）、alpha矩阵、beta矩阵
    输出：gamma矩阵
    '''
    gamma = np.zeros(alpha.shape) #gamma的维度时和alpha一样的
    gamma = alpha * beta #矩阵对应元素相乘
    gamma = gamma / np.sum(gamma,0)
    return gamma

def getXi(hmm, observations, alpha, beta):
    '''
    计算xi矩阵
    输入：HMM模型（包含初始向量pi，转移矩阵A、发射矩阵B）、观察序列、alpha矩阵、beta矩阵
    输出：xi矩阵
    '''
    colNum = len(observations)
    rowNum = hmm.statesNumber
    xi = np.zeros((rowNum, rowNum, colNum-1))

    for t in range(0, colNum-1):        
        for i in range(0, rowNum):
            for j in range(0, rowNum):
                xi[i,j,t] = alpha[i,t] * hmm.transitionMatrix[i,j] * beta[j,t+1] * \
                            hmm.emissionMatrix[j,observations[t+1]-1] 
        xi[:,:,t] /= np.sum(xi[:,:,t])    #modify
    return xi

def baumWelch(hmm, observations):
     pass
    
#todo

## 关于argmax函数

这个函数有点不熟悉，这里备注下使用方法。

**numpy.argmax(a, axis=None, out=None) ** ： 返回沿轴axis最大值的索引。

**Parameters**: 
a : array_like 数组 
axis : int, 可选，默认情况下，索引的是平铺的数组，否则沿指定的轴。 
out : array, 可选，如果提供，结果以合适的形状和类型被插入到此数组中。 

**Returns**: 
index_array : ndarray of ints,索引数组。它具有与a.shape相同的形状，其中axis被移除。 

例子：

In [8]:
import numpy as np
a = np.array([[0, 1, 2],
              [3, 4, 5]])
np.argmax(a)

5

In [9]:
np.argmax(a, axis=0)#0代表列

array([1, 1, 1], dtype=int64)

In [10]:
np.argmax(a, axis=1)#1代表行

array([2, 2], dtype=int64)

In [11]:
b = np.array([0, 5, 2, 3, 4, 5])
np.argmax(b) # 只返回第一次出现的最大值的索引

1